# Generate Gcode to Resurface Wasteboard

In [1]:
from GCodify import GCodify     # custom library to generate gcode
import numpy as np

# Machine Settings

In [13]:
x_limits = [0, -280.0]           # machine table limits
y_limits = [0, -172.0]
z_safe = 2.0                    # safe z position for rapid moves
z_plunge = -0.5                  # z cutting position
cutter_diameter_mm  = 24.0      # diameter of surface cutter in mm
cutter_overlap_percent = 25     # percent overlap of cutter path
spindle_speed = 12000           # machine spindle speed

plunge_feedrate = 20            # z plunge feedrate in mm/min
cutting_feedrate = 200          # xy cutting feedrate in mm/min

fname = "spoilboard_resurface_V2.gcode"

# Gcode generation for cutting right to left

In [16]:
gcode = GCodify()

# add gcode header
gcode.comment("GCODE HEADER")
gcode.custom("G21 ; set units to mm")
gcode.custom("G90 ; set distance mode to absolute positioning ")
gcode.custom("G94 ; set feedrate mode to mm/min ")

# move to starting position
gcode.goto_z(z_safe, "goto safe z position for rapid moves")
gcode.goto_xy(max(x_limits), max(y_limits), "goto min xy position ")
gcode.custom("M03 S{} ; turn on spindle".format(spindle_speed))
gcode.comment(" ")

cutter_overlap_mm = cutter_diameter_mm * (cutter_overlap_percent / 100)
y_step = cutter_diameter_mm - cutter_overlap_mm

# start cutting right to left
gcode.comment("GCODE BODY")
for y in np.arange(max(y_limits), min(y_limits), -1*y_step):
    gcode.goto_xy(max(x_limits), y)
    gcode.move_z(z_plunge, plunge_feedrate)
    gcode.move_xy(min(x_limits), y, cutting_feedrate)
    gcode.goto_z(z_safe)

# final pass right to left
gcode.goto_xy(max(x_limits), min(y_limits))
gcode.move_z(z_plunge, plunge_feedrate)
gcode.move_xy(min(x_limits), min(y_limits), cutting_feedrate)
gcode.goto_z(z_safe)

# start cutting top to bottom
gcode.goto_z(z_safe)
gcode.goto_xy(max(x_limits), max(y_limits))
for x in np.arange(max(x_limits), min(x_limits), -1*y_step):
    gcode.goto_xy(x, max(y_limits))
    gcode.move_z(z_plunge, plunge_feedrate)
    gcode.move_xy(x, min(y_limits), cutting_feedrate)
    gcode.goto_z(z_safe)

# final pass top to bottom
gcode.goto_xy(min(x_limits), max(y_limits))
gcode.move_z(z_plunge, plunge_feedrate)
gcode.move_xy(min(x_limits), min(y_limits), cutting_feedrate)
gcode.goto_z(z_safe)

# add gcode footer
gcode.comment(" ")
gcode.comment("GCODE FOOTER")
gcode.goto_z(z_safe)
gcode.custom("M05 ; turn off spindle ")
gcode.goto_xy(max(x_limits), max(y_limits))

gcode.pretty()  # align left side of all inline comments
gcode.write(fname)  # write gcode file